In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [48]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random


In [377]:
import numpy as np
path_to_data = "/gdrive/MyDrive/MIE1666project/rl-ccmvo-main/rl-ccmvo-main/Data/synthetic1000"
date = "2016-02-07"
mu = np.loadtxt('{}/{}/mu'.format(path_to_data, date))
Q = np.loadtxt('{}/{}/Q'.format(path_to_data, date))

In [308]:
import cvxpy as cp
import sympy as sp
def MVO(mu,Q,gamma):
    # Convert mu to NumPy array if it's not already
  if not isinstance(mu, np.ndarray):
      mu = np.array(mu)

  # Convert Q to NumPy array if it's not already
  if not isinstance(Q, np.ndarray):
      Q = np.array(Q)
  if mu.size == 0 or Q.size == 0:
    return 0,0
  else:
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    constraints = [
        cp.sum(w) == 1, # Sum to 1
        w>=0 # Disallow Short Sales
    ]

    # Objective Function
    risk = cp.quad_form(w, Q)
    targetRet=gamma*mu.T@w
    prob = cp.Problem(cp.Maximize(targetRet - risk), constraints=constraints)
    prob.solve()
    return w.value,targetRet.value-risk.value

In [386]:
# function to select Qs from the Q array of 1000/500 el
def generate_Q(Q,index):
  # Use np.meshgrid to generate all pairs of indices
  all_pairs = np.array(np.meshgrid(index,index)).T.reshape(-1, 2)

  # Use advanced indexing to select the elements
  selected_elements = Q[all_pairs[:, 0], all_pairs[:, 1]]
  size=int(np.sqrt(selected_elements.size))
  return selected_elements.reshape(size,size)


In [1274]:
Q=([3,3,4],[1,2,6])
Q[1][-1]

6

In [1285]:
# derivative function always return the gradient of MVO wrt the last x added to the list
def derivative(mu,Q,gamma,w):
  # if mu.size <= 1 or Q.size <= 1:
  #   return -gamma*mu[0]
  # else:
  risk=0
  Ret=0
  for i in range(len(mu)):
      risk+=Q[i][-1]*w[i]
      Ret+=mu[-1]
      # Ret+=mu[-1]*gamma
  return Ret-risk


In [1286]:
class TrainEnv(Env):
    def __init__(self, Q):
        self.index = []
        self.cur_mu = []
        self.cur_Q = []
        self.action_space = Discrete(1000)
        self.observation_space = Box(low=np.array(-np.inf), high=np.array(np.inf))
        self.state = []
        self.asset_length = 50
        self.used_actions = set()  # Keep track of used actions in the current iteration

    def step(self, action):
        # action is the index of the asset
        action = self.sample_unique_action()
        self.used_actions.add(action)
        gamma = 1
        self.index.append(action)
        self.cur_mu.append(mu[action])  # Assuming mu is defined somewhere
        self.cur_Q=generate_Q(Q,self.index) # Assuming generate_Q is defined
        w, self.state = MVO(self.cur_mu, self.cur_Q, 1)  # Assuming MVO is defined
        self.asset_length -= 1
        self.transition = derivative(self.cur_mu,self.cur_Q, gamma, w)  # Assuming derivative is defined

        # Calculate reward
        if self.transition > 0:
            reward = 1
        else:
            reward = -1

        done = self.asset_length <= 0

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    # Make sure each action are unique
    def sample_unique_action(self):
      # Sample a new action until it is unique in the current iteration
      new_action = self.action_space.sample()
      while new_action in self.used_actions:
          new_action = self.action_space.sample()
      return new_action
    def render(self):
        # Implement viz
        pass

    def reset(self):
        # Reset
        self.state = []
        self.used_actions.clear()
        self.asset_length = 50
        return self.state


In [1287]:
env=TrainEnv(Q)

In [1288]:
env.reset()

[]

515

In [1289]:
action=env.action_space.sample()
env.step(action)

TypeError: ignored

In [507]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info= env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

DCPError: ignored

In [ ]:
import numpy as np
path_to_data = "/gdrive/MyDrive/MIE1666project/rl-ccmvo-main/rl-ccmvo-main/Data/synthetic1000"
date = "2016-02-07"
mu = np.loadtxt('{}/{}/mu'.format(path_to_data, date))
Q = np.loadtxt('{}/{}/Q'.format(path_to_data, date))

In [508]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.2.0+cu92 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1

In [509]:
import torch
from torch.autograd import Variable

In [510]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# pip install "ray[rllib]" tensorflow torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.6 MB/s eta 0:00:00


In [ ]:
# from ray.rllib.algorithms.dqn.dqn import DQNConfig

/usr/local/lib/python3.10/dist-packages/flax/configurations.py:42: DeprecationWarning: jax.config.define_bool_state is deprecated. Please use other libraries for configuration instead.
  return jax_config.define_bool_state('flax_' + name, default, help)
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated

In [ ]:
# config = DQNConfig()
# print(config.replay_buffer_config)
# replay_config = config.replay_buffer_config.update(
#     {
#         "capacity": 60000,
#         "prioritized_replay_alpha": 0.5,
#         "prioritized_replay_beta": 0.5,
#         "prioritized_replay_eps": 3e-6,
#     }
# )
# config = config.training(replay_buffer_config=replay_config)
# config = config.resources(num_gpus=1)
# config = config.rollouts(num_rollout_workers=3)
# algo = config.build()
# algo.train()

{'type': 'MultiAgentPrioritizedReplayBuffer', 'prioritized_replay': -1, 'capacity': 50000, 'prioritized_replay_alpha': 0.6, 'prioritized_replay_beta': 0.4, 'prioritized_replay_eps': 1e-06, 'replay_sequence_length': 1, 'worker_side_prioritization': False}


AssertionError: ignored

In [511]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import gym

In [519]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000
# env = gym.make('CartPole-v0')
# env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndexError: ignored

In [518]:
ENV_A_SHAPE

0

In [515]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 10)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(10, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [516]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [517]:
dqn = DQN()

TypeError: ignored

In [ ]:
a = dqn.choose_action(s)
len(env.step(a))
# s, r, done, info = env.step(a)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


5

In [ ]:

dqn = DQN()

print('\nCollecting experience...')
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    while True:
        a = dqn.choose_action(s)
        s_, r, done, info,_= env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_


Ep:  200 | Ep_r:  1.59
Ep:  201 | Ep_r:  1.66
Ep:  202 | Ep_r:  7.25
Ep:  203 | Ep_r:  1.77
Ep:  204 | Ep_r:  2.89
Ep:  205 | Ep_r:  1.15
Ep:  206 | Ep_r:  0.95
Ep:  207 | Ep_r:  3.86
Ep:  208 | Ep_r:  2.39
Ep:  209 | Ep_r:  -0.06
Ep:  210 | Ep_r:  9.1
Ep:  211 | Ep_r:  2.81
Ep:  212 | Ep_r:  9.43
Ep:  213 | Ep_r:  15.44
Ep:  214 | Ep_r:  3.87
Ep:  215 | Ep_r:  1.95
Ep:  216 | Ep_r:  39.42
Ep:  217 | Ep_r:  17.42
Ep:  218 | Ep_r:  13.74
Ep:  219 | Ep_r:  14.23
Ep:  220 | Ep_r:  13.3
Ep:  221 | Ep_r:  28.49
Ep:  222 | Ep_r:  65.93
Ep:  223 | Ep_r:  33.98
Ep:  224 | Ep_r:  173.3
Ep:  225 | Ep_r:  52.96
Ep:  226 | Ep_r:  89.72
Ep:  227 | Ep_r:  90.59
Ep:  228 | Ep_r:  141.99
Ep:  229 | Ep_r:  110.4
Ep:  230 | Ep_r:  948.68
Ep:  231 | Ep_r:  503.84
Ep:  232 | Ep_r:  319.7
Ep:  233 | Ep_r:  311.24
Ep:  234 | Ep_r:  353.37
Ep:  235 | Ep_r:  424.86
Ep:  236 | Ep_r:  262.47
Ep:  237 | Ep_r:  462.2
Ep:  238 | Ep_r:  406.6
Ep:  239 | Ep_r:  329.02
Ep:  240 | Ep_r:  216.19
Ep:  241 | Ep_r:  340.